In [ ]:
#==================== We set product parameters to download ====================
destination_path = '../GOESimages/'
product_list = { # ABI Products
"ABI-L2-ACHAF": "Cloud Top Height",
"ABI-L2-ACHTF": "Cloud Top Temperature",
"ABI-L2-ACMF": "Clear Sky  Mask",
"ABI-L2-ACTPF": "Cloud Top Phase",
## "ABI-L2-DMWVF": "Derived Motion Winds - Vapor",
## "ABI-L2-DSRF": "Downward Shortwave Radiation",
"ABI-L2-LSTF": "Land Surface Temperature",
"ABI-L2-RRQPEF": "Rainfall rate",
"ABI-L2-TPWF": "Total Precipitable Water",
}
products = list(product_list)
#==================== Setting up time reference variables ====================
from datetime import datetime, timedelta, timezone
import os, pytz, time
utc = pytz.timezone('UTC') # UTC timezone
utcm5 = pytz.timezone('America/Lima') # UTC-5 timezone
from IPython.display import display, Image, clear_output
import numpy as np
import GOESutils.GOESplots as gplt
import GOESutils.GOESimport as gimp
import GOESutils.DataBaseUtils as dbu
import goes2go as g2g
import importlib, sys
# importlib.reload(sys.modules['GOESutils.GOESplots'])
# import GOESutils.GOESplots as gplt
# importlib.reload(sys.modules['GOESutils.GOESimport'])
# import GOESutils.GOESimport as gimp

# Products in a time range

In [ ]:
importlib.reload(sys.modules['GOESutils.GOESimport'])
import GOESutils.GOESimport as gimp

In [ ]:
# importlib.reload(sys.modules['GOESutils.GOESimport'])
# import GOESutils.GOESimport as gimp
startdate = datetime(2020,1,1,tzinfo=utcm5)
enddate = datetime(2020,2,1,tzinfo=utcm5)
prodFileList = gimp.GOESfiles("timerange", startdate, enddate, freq=timedelta(minutes=30),
                              target_product="RadF", bands=14,
                              to_display=True, download=False)

In [ ]:
initial_time = datetime.now(utcm5)
product = prodFileList['product'][0]
for f in list(prodFileList['file']): # Reading each file downloaded
    print("Working with file: {}".format(os.path.basename(f)))                
    FullFileName = os.path.join(destination_path,f)
    data, ProductParams = gimp.ImportingData(FullFileName, product)
    data = gimp.CleaningData(data, product)
    dataset = gplt.export_as(data, ProductParams["DataAttrs"], 
                        filename=ProductParams["DataAttrs"]['dataset_name'],
                        path="./GOESimages/Data")
    

    # Check if it's time to clear the output
    CurrentTime = datetime.now(utcm5)
    minute, seconds = CurrentTime.minute, CurrentTime.second
    elapsed_time = CurrentTime - initial_time
    if elapsed_time >= timedelta(minutes=1):
        # Clear the output
        clear_output(wait=True)
        # Reset the start time
        initial_time = datetime.now(utcm5)
        
print("All the files have been processed.")

In [ ]:
initial_time = datetime.now(utcm5)
for product in products:
    for f in list(prodFileList['file']): # Reading each file downloaded
        print("Working with file: {}".format(os.path.basename(f)))                
        FullFileName = os.path.join(destination_path,f)
        data, ProductParams = gimp.ImportingData(FullFileName, product)
        data = gimp.CleaningData(data, product)
        FullImageName = os.path.join(ProductParams["ImagePath"],"Peru",ProductParams["ImageName"])
        if os.path.exists(FullImageName): # If png image exists, it is shown
            print("Image '{}' already exists in '{}'".format(ProductParams["ImageName"],ProductParams["ImagePath"]))
            display(Image(filename=FullImageName, width=640))
        else: # Creating png image
            file_datetime = ProductParams["ImageTime"].astimezone(utc)
            gFileList = goes_nearesttime(file_datetime.replace(tzinfo=None), product="ABI-L2-MCMIPF", return_as="filelist", download=True, save_dir=destination_path)
            try:
                RGBdata, GeoColorParams = gimp.GeoColorData(os.path.join(destination_path,gFileList.iloc[0]["file"]))
            except:
                gFileList = goes_nearesttime(file_datetime.replace(tzinfo=None), product="ABI-L2-MCMIPF", return_as="filelist", download=True, save_dir=destination_path, overwrite=True)
                RGBdata, GeoColorParams = gimp.GeoColorData(os.path.join(destination_path,gFileList.iloc[0]["file"]))
            figGeo, axGeo = gplt.GeoColorPlot(RGBdata, GeoColorParams, toSave=False, toDisplay=False, toUpload=False, dpi=300)
            if (not os.path.exists(ProductParams["ImagePath"])):
                print(f"Directory for product {product} does not exist. Creating new one...") 
                os.makedirs(ProductParams["ImagePath"])
            print(f"Image for file {os.path.basename(f)} not found, creating one...")
            figProd = gplt.ProductPlot(data, product, axGeo, ProductParams, toSave=True, toDisplay=True, toUpload=False, dpi=600)

        # Check if it's time to clear the output
        CurrentTime = datetime.now(utcm5)
        minute, seconds = CurrentTime.minute, CurrentTime.second
        elapsed_time = CurrentTime - initial_time
        if elapsed_time >= timedelta(minutes=1):
            # Clear the output
            clear_output(wait=True)
            # Reset the start time
            initial_time = datetime.now(utcm5)
            
    print("All the files have been processed.")

In [ ]:
importlib.reload(sys.modules['GOESutils.GOESplots'])
import GOESutils.GOESplots as gplt
ImagesPath = ".\GOESimages\Products\ACHA\Peru"
ImagesInfo = gplt.GettingImagesInfo(ImagesPath, start_date=startdate, end_date=enddate)
gplt.GOESvideos(ImagesInfo, VideoPath=".\GOESimages\Products\ACHA", extension=".avi")

# Products in real time

In [ ]:
# importlib.reload(sys.modules['GOESutils.GOESimport'])
# import GOESutils.GOESimport as gimp
# importlib.reload(sys.modules['GOESutils.GOESplots'])
# import GOESutils.GOESplots as gplt

toUpload = False
while True:
    clear_output(wait=True)
    CurrentTime = datetime.now(utcm5)
    CurrentTime_str = CurrentTime.strftime('%Y-%m-%d %H:%M:%S %Z')
    print("============================================================")
    print("Current time is: {}".format(CurrentTime_str))
    # gFileList = gimp.GOESfiles("latest", target_product="ABI-L2-MCMIP", download=True)
    # f = gFileList.iloc[0]
    # RGBdata, GeoColorParams = gimp.GeoColorData(os.path.join(destination_path,f["file"]))
    RGBdata, GeoColorParams = gplt.GeoColorTif(destination_path)
    figGeo, axGeo = gplt.GeoColorPlot(RGBdata, GeoColorParams, toSave=True, toDisplay=True, toUpload=False, dpi=150)
    prodFileList = gimp.GOESfiles("latest", target_product=products, to_display=True, download=True)
    for product in products:
        
        f = prodFileList[prodFileList["product"]==product].iloc[0]
        FullFileName = os.path.join(destination_path,f["file"])
        data, ProductParams = gimp.ImportingData(FullFileName, product)
        data = gimp.CleaningData(data, product)
        data_re = gimp.interpolate_products(data, product, n=5)
                                                    
        print("Working with file: {}".format(os.path.basename(f['file'])))                
        FullImageName = os.path.join(ProductParams["ImagePath"],"Peru",ProductParams["ImageName"])
        if os.path.exists(FullImageName): # If png image exists, it is shown
            print("Image [{}] already exists in [{}]".format(ProductParams["ImageName"],ProductParams["ImagePath"]))
            display(Image(filename=FullImageName)) # , width=540   
        else: # Creating png image
            if not os.path.exists(ProductParams["ImagePath"]):
                print(f"Directory for product {product} does not exist. Creating new one...") 
                os.makedirs(ProductParams["ImagePath"])
            print(f"Image for file {os.path.basename(f['file'])} not found, creating one...")
            figProd = gplt.ProductPlot(data_re, product, axGeo, ProductParams, toSave=True, toDisplay=True, toUpload=toUpload, dpi=150)

        # # try: # Deleting downloaded product
        # #     os.remove(FullFileName)
        # #     print(f"File '{FullFileName}' has been removed.")
        # # except FileNotFoundError:
        # #     print(f"File '{FullFileName}' not found.")
        # # except Exception as e:
        # #     print(f"An error occurred while deleting the file: {e}")
            
        for dep in gplt.departments[:3]:
            gplt.DepartmentPlot(product, dep, RGBdata, GeoColorParams, data_re, ProductParams, toSave=True, toDisplay=True, toUpload=toUpload)
        reports = gplt.ReportingEvents(data, product, level="L3", send_comments=True)
            
        print("All the files have been processed.")
        

    try: # Check if it's time to clear the output
        CurrentTime = datetime.now(utcm5)
        hour, minute, seconds = CurrentTime.hour, CurrentTime.minute, CurrentTime.second
        if (hour==23 and minute>55): # (elapsed_time >= timedelta(hours=3)) or 
            print("Cleaning database")
            for i in range(2,9):
                folder = f"Images/Type{i}"
                dbu.DeleteFile(folder)
                for d in gplt.departments_folder:
                    folder = f"Images/T{i}{d}"
                    dbu.DeleteFile(folder)
            
        total_remaining_seconds = (5 - (int(minute) % 5)) * 60 - int(seconds)
        remaining_minutes = total_remaining_seconds // 60
        print("Waiting {} minutes for the next file upload".format(remaining_minutes + 2))
        time.sleep((remaining_minutes+2)*60)
    except Exception as e:
        print("Elapsed time attempting failed.")